# HISTORY

#### Ethnicity Labels

ETHNICITIES = { 
    0: "White", 
    1: "Black",
    2: "Asian",
    3: "Indian",
    4: "Hispanic"
}

#### Gender Labels

GENDERS = { 
    0: "Male", 
    1: "Female"
}


#### What Will You See

* Effect of various loss functions on similar models
* Different data preparation stages for different models
* Data preparation processes
* Various models
* Last model for ethnicities


##### Please follow carefully

# PACKAGES AND LIBRARIES

In [ ]:
#GENERAL
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random
#PATH PROCESS
import os
import os.path
from pathlib import Path
import glob
#IMAGE PROCESS
from PIL import Image
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
from keras.applications.vgg16 import preprocess_input, decode_predictions
from keras.preprocessing import image
import skimage
from skimage.feature import hessian_matrix, hessian_matrix_eigvals
from scipy.ndimage.filters import convolve
from skimage import data, io, filters
#SCALER & TRANSFORMATION
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from keras import regularizers
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
#ACCURACY CONTROL
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score, roc_curve
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
#OPTIMIZER
from keras.optimizers import RMSprop,Adam,Optimizer,Optimizer, SGD
#MODEL LAYERS
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization,MaxPooling2D,BatchNormalization,\
                        Permute, TimeDistributed, Bidirectional,GRU, SimpleRNN, LSTM, GlobalAveragePooling2D, SeparableConv2D,\
ZeroPadding2D, Convolution2D, ZeroPadding2D,AveragePooling2D
from keras import models
from keras import layers
import tensorflow as tf
from keras.applications import VGG16,VGG19,inception_v3
from keras import backend as K
from keras.utils import plot_model
from keras.models import load_model
from keras.regularizers import l1,l2,L1L2
from tensorflow.keras import regularizers
#SKLEARN CLASSIFIER
from xgboost import XGBClassifier, XGBRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.cross_decomposition import PLSRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV
#IGNORING WARNINGS
from warnings import filterwarnings
filterwarnings("ignore",category=DeprecationWarning)
filterwarnings("ignore", category=FutureWarning) 
filterwarnings("ignore", category=UserWarning)

# PATH, LABEL, TRANSFORMATION

#### MAIN CSV

In [ ]:
Main_G_A_E_Path = pd.read_csv("../input/age-gender-and-ethnicity-face-data-csv/age_gender.csv")

In [ ]:
print(Main_G_A_E_Path.head(-1))

#### CSV RE-BUILT

In [ ]:
Main_G_A_E_Path = Main_G_A_E_Path.drop("img_name",axis=1)

In [ ]:
print(Main_G_A_E_Path.head())

In [ ]:
Main_G_A_E_Path["age"] = Main_G_A_E_Path["age"].apply(lambda x: "Teen" if x < 18 else("Middle" if 18 <= x < 55 else "Old"))

In [ ]:
print(Main_G_A_E_Path["age"].value_counts())

In [ ]:
print(Main_G_A_E_Path.head(-1))

In [ ]:
Encoder_Function = LabelEncoder()

In [ ]:
print("MAIN:")
print(Main_G_A_E_Path["age"].value_counts())
print("---" * 20)
Main_G_A_E_Path["age"] = Encoder_Function.fit_transform(Main_G_A_E_Path["age"])
print("TRANSFORMED:")
print(Main_G_A_E_Path["age"].value_counts())

In [ ]:
print(Main_G_A_E_Path.head())

In [ ]:
Main_G_A_E_Path["pixels"] = Main_G_A_E_Path["pixels"].apply(lambda x: np.array(x.split(),dtype="float32"))
Main_G_A_E_Path["pixels"] = Main_G_A_E_Path["pixels"] / 255.

In [ ]:
print(Main_G_A_E_Path.head())

In [ ]:
print("Single Image Shape:")
print(Main_G_A_E_Path["pixels"][0].shape)
print("Total Shape:")
print(Main_G_A_E_Path["pixels"].shape)

In [ ]:
X_Train = np.zeros(shape=(23705,2304))

for indexing in range(len(Main_G_A_E_Path["pixels"])):
    X_Train[indexing] = Main_G_A_E_Path["pixels"][indexing]

In [ ]:
print(X_Train.shape)

In [ ]:
# "2304 = 48*48"

X_Train = X_Train.reshape(-1,48,48,1)

In [ ]:
print(X_Train.shape)

In [ ]:
print(X_Train[0].shape)

In [ ]:
print(type(X_Train))

# VISION

In [ ]:
figure,axis = plt.subplots(5,5,figsize=(10,10))

for indexing,operations in enumerate(axis.flat):
    Picking_Image = X_Train[indexing]
    
    operations.set_xlabel(f"AGE: {str(Main_G_A_E_Path.age.iloc[indexing])} "+f"ETH: {str(Main_G_A_E_Path.ethnicity.iloc[indexing])} "+f"GENDER: {str(Main_G_A_E_Path.gender.iloc[indexing])}")
    operations.imshow(Picking_Image)
    
plt.tight_layout()
plt.show()

In [ ]:
figure = plt.figure(figsize=(10,10))

random_picking = index=np.random.randint(0,23000,16)

for indexing in range(len(random_picking)):
    plt.subplot(4,4,(indexing%25)+1)
    plt.grid(True)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(X_Train[random_picking[indexing]])
    
    plt.xlabel(f"AGE: {str(Main_G_A_E_Path.age.iloc[random_picking[indexing]])} "+f"ETH: {str(Main_G_A_E_Path.ethnicity.iloc[random_picking[indexing]])} "+f"GENDER: {str(Main_G_A_E_Path.gender.iloc[random_picking[indexing]])}")

plt.show()

# DATA PROCESS FOR PREDICTION

#### TO ARRAY

In [ ]:
Main_G_A_E_Path["age"] = np.array(Main_G_A_E_Path["age"],dtype="float32")
Main_G_A_E_Path["gender"] = np.array(Main_G_A_E_Path["gender"],dtype="float32")
Main_G_A_E_Path["ethnicity"] = np.array(Main_G_A_E_Path["ethnicity"],dtype="float32")

#### SPLITTING

In [ ]:
X = X_Train # pixels
Y = Main_G_A_E_Path.iloc[:,:-1].values # prediction targets

In [ ]:
print(X.shape)
print(Y.shape)

In [ ]:
print(type(X))
print(type(Y))

* first column = age
* second column = ethnicity
* third column = gender

In [ ]:
print(Y)

In [ ]:
xTrain,xTest,yTrain,yTest = train_test_split(X,Y,train_size=0.9,random_state=42,shuffle=True)

In [ ]:
print(xTrain.shape)
print(xTest.shape)
print(yTrain.shape)
print(yTest.shape)

# GENERAL PARAMETERS

In [ ]:
Early_Stopper = tf.keras.callbacks.EarlyStopping(monitor="loss",patience=5,mode="min")
Checkpoint_Model = tf.keras.callbacks.ModelCheckpoint(monitor="val_accuracy",
                                                      save_best_only=True,
                                                      save_weights_only=True,
                                                      filepath="./modelcheck")

In [ ]:
# for age
A_COMPILE_OPTIMIZER = Adam()
A_COMPILE_LOSS = "msle" # mean squared logarithmic error
A_COMPILE_METRICS = ["accuracy"]
A_INPUT_SHAPE = (xTrain.shape[1],xTrain.shape[2],xTrain.shape[3])
A_OUTPUT_CLASS = 1

In [ ]:
# for ethnicity
E_COMPILE_OPTIMIZER = Adam()
E_COMPILE_LOSS = "mean_absolute_percentage_error" # mean squared logarithmic error
E_COMPILE_METRICS = ["accuracy"]
E_INPUT_SHAPE = (xTrain.shape[1],xTrain.shape[2],xTrain.shape[3])
E_OUTPUT_CLASS = 1

In [ ]:
# for gender
G_COMPILE_OPTIMIZER = Adam()
G_COMPILE_LOSS = "categorical_hinge" # mean squared logarithmic error
G_COMPILE_METRICS = ["accuracy"]
G_INPUT_SHAPE = (xTrain.shape[1],xTrain.shape[2],xTrain.shape[3])
G_OUTPUT_CLASS = 1

In [ ]:
# for general
COMPILE_OPTIMIZER = Adam()
COMPILE_LOSS = "msle" # mean squared logarithmic error
COMPILE_METRICS = ["accuracy"]
INPUT_SHAPE = (xTrain.shape[1],xTrain.shape[2],xTrain.shape[3])
OUTPUT_CLASS = 3

In [ ]:
# for last
L_COMPILE_OPTIMIZER = Adam()
L_COMPILE_LOSS = "categorical_crossentropy" # mean squared logarithmic error
L_COMPILE_METRICS = ["accuracy"]
L_INPUT_SHAPE = (xTrain.shape[1],xTrain.shape[2],xTrain.shape[3])
L_OUTPUT_CLASS = 5

# MODEL FOR ALL

In [ ]:
Model = Sequential()

Model.add(Conv2D(32,(3,3),activation="relu",input_shape=INPUT_SHAPE))
Model.add(BatchNormalization())
Model.add(MaxPooling2D((2,2)))

Model.add(Conv2D(64,(3,3),activation="relu"))
Model.add(Dropout(0.3))
Model.add(MaxPooling2D((2,2)))
Model.add(BatchNormalization())

Model.add(Conv2D(128,(3,3),activation="relu"))
Model.add(Dropout(0.3))
Model.add(MaxPooling2D((2,2)))
Model.add(BatchNormalization())


Model.add(Flatten())
Model.add(Dense(128,activation="relu"))
Model.add(BatchNormalization())
Model.add(Dropout(0.5))
Model.add(Dense(64,activation="relu"))
Model.add(BatchNormalization())
Model.add(Dropout(0.5))
Model.add(Dense(OUTPUT_CLASS,activation="softmax"))

In [ ]:
Model.compile(optimizer=COMPILE_OPTIMIZER,loss=COMPILE_LOSS,metrics=COMPILE_METRICS)

In [ ]:
CNN_Model = Model.fit(xTrain,yTrain,
                      validation_data=(xTest,yTest),
                      callbacks=[Early_Stopper,Checkpoint_Model],
                      batch_size=12,
                      epochs=50)

In [ ]:
Model_Results = Model.evaluate(xTest,yTest)
print("LOSS:  " + "%.4f" % Model_Results[0])
print("ACCURACY:  " + "%.2f" % Model_Results[1])

In [ ]:
Grap_Data = pd.DataFrame(CNN_Model.history)
Grap_Data.plot()

# MODEL FOR AGE

#### RE-BUILT

In [ ]:
print(yTrain.shape)
print(yTest.shape)

In [ ]:
print(yTrain[:,:1].shape) # for age
print(yTest[:,:1].shape)

In [ ]:
A_yTrain = yTrain[:,:1]
A_yTest = yTest[:,:1]

#### STRUCTURE

In [ ]:
Model_A = Sequential()

Model_A.add(Conv2D(32,(3,3),activation="relu",input_shape=A_INPUT_SHAPE))
Model_A.add(BatchNormalization())
Model_A.add(MaxPooling2D((2,2)))

Model_A.add(GlobalAveragePooling2D())
Model_A.add(Flatten())
Model_A.add(Dense(128,activation="relu"))
Model_A.add(BatchNormalization())
Model_A.add(Dropout(0.5))
Model_A.add(Dense(64,activation="relu"))
Model_A.add(BatchNormalization())
Model_A.add(Dropout(0.5))
Model_A.add(Dense(A_OUTPUT_CLASS,activation="sigmoid"))

In [ ]:
Model_A.compile(optimizer=A_COMPILE_OPTIMIZER,loss=A_COMPILE_LOSS,metrics=A_COMPILE_METRICS)

In [ ]:
A_CNN_Model = Model_A.fit(xTrain,A_yTrain,
                      validation_data=(xTest,A_yTest),
                      callbacks=Early_Stopper,
                      batch_size=12,
                      epochs=50)

In [ ]:
Model_Results = Model_A.evaluate(xTest,A_yTest)
print("LOSS:  " + "%.4f" % Model_Results[0])
print("ACCURACY:  " + "%.2f" % Model_Results[1])

In [ ]:
Grap_Data = pd.DataFrame(A_CNN_Model.history)
Grap_Data.plot()

# MODEL FOR ETHNICITY

#### RE-BUILT

In [ ]:
print(yTrain)

In [ ]:
print(yTrain[:,1:2]) # for ethnicity

In [ ]:
print(yTrain[:,1:2].shape)

In [ ]:
E_yTrain = yTrain[:,1:2]
E_yTest = yTest[:,1:2]

#### STRUCTURE

In [ ]:
Model_E = Sequential()

Model_E.add(Conv2D(32,(3,3),activation="relu",input_shape=E_INPUT_SHAPE))
Model_E.add(BatchNormalization())
Model_E.add(MaxPooling2D((2,2)))

Model_E.add(GlobalAveragePooling2D())
Model_E.add(Flatten())
Model_E.add(Dense(128,activation="relu"))
Model_E.add(BatchNormalization())
Model_E.add(Dropout(0.5))
Model_E.add(Dense(64,activation="relu"))
Model_E.add(BatchNormalization())
Model_E.add(Dropout(0.5))
Model_E.add(Dense(E_OUTPUT_CLASS,activation="sigmoid"))

In [ ]:
Model_E.compile(optimizer=E_COMPILE_OPTIMIZER,loss=E_COMPILE_LOSS,metrics=E_COMPILE_METRICS)

In [ ]:
E_CNN_Model = Model_E.fit(xTrain,E_yTrain,
                      validation_data=(xTest,E_yTest),
                      callbacks=Early_Stopper,
                      batch_size=12,
                      epochs=50)

In [ ]:
Model_Results = Model_E.evaluate(xTest,E_yTest)
print("LOSS:  " + "%.4f" % Model_Results[0])
print("ACCURACY:  " + "%.2f" % Model_Results[1])

In [ ]:
Grap_Data = pd.DataFrame(E_CNN_Model.history)
Grap_Data.plot()

# MODEL FOR GENDER

#### RE-BUILT

In [ ]:
G_yTrain = yTrain[:,2:3]
G_yTest = yTest[:,2:3]

#### STRUCTURE

In [ ]:
Model_G = Sequential()

Model_G.add(Conv2D(32,(3,3),activation="relu",input_shape=G_INPUT_SHAPE))
Model_G.add(BatchNormalization())
Model_G.add(MaxPooling2D((2,2)))

Model_G.add(Conv2D(64,(3,3),activation="relu"))
Model_G.add(Dropout(0.3))
Model_G.add(MaxPooling2D((2,2)))
Model_G.add(BatchNormalization())

Model_G.add(Conv2D(128,(3,3),activation="relu"))
Model_G.add(Dropout(0.3))
Model_G.add(MaxPooling2D((2,2)))
Model_G.add(BatchNormalization())


Model_G.add(Flatten())
Model_G.add(Dense(128,activation="relu"))
Model_G.add(BatchNormalization())
Model_G.add(Dropout(0.5))
Model_G.add(Dense(64,activation="relu"))
Model_G.add(BatchNormalization())
Model_G.add(Dropout(0.5))
Model_G.add(Dense(G_OUTPUT_CLASS,activation="softmax"))

In [ ]:
Model_G.compile(optimizer=G_COMPILE_OPTIMIZER,loss=G_COMPILE_LOSS,metrics=G_COMPILE_METRICS)

In [ ]:
G_CNN_Model = Model_G.fit(xTrain,G_yTrain,
                      validation_data=(xTest,G_yTest),
                      callbacks=Early_Stopper,
                      batch_size=12,
                      epochs=50)

In [ ]:
Model_Results = Model_G.evaluate(xTest,G_yTest)
print("LOSS:  " + "%.4f" % Model_Results[0])
print("ACCURACY:  " + "%.2f" % Model_Results[1])

In [ ]:
Grap_Data = pd.DataFrame(G_CNN_Model.history)
Grap_Data.plot()

# LAST MODEL FOR ETHNICITY

#### RE-BUILT

In [ ]:
X_Train = np.zeros(shape=(23705,2304))

for indexing in range(len(Main_G_A_E_Path["pixels"])):
    X_Train[indexing] = Main_G_A_E_Path["pixels"][indexing]

In [ ]:
X = X_Train # pixels
Y = Main_G_A_E_Path.iloc[:,:-1] # prediction targets

In [ ]:
print(X.shape)
print(Y.shape)

In [ ]:
print(Y)

In [ ]:
Y.ethnicity=Y.ethnicity.astype('object')

In [ ]:
print(Y.ethnicity)

In [ ]:
Y_ethnicity=np.array(to_categorical(Y.ethnicity))

In [ ]:
print("Ethnicity Labels:",Y_ethnicity.shape)

In [ ]:
X = X_Train.reshape(-1,48,48,1)

In [ ]:
print(X.shape)

In [ ]:
xTrain,xTest,yTrain,yTest = train_test_split(X,Y_ethnicity,train_size=0.9,random_state=42,shuffle=True)

In [ ]:
print(xTrain.shape)
print(xTest.shape)
print(yTest.shape)
print(yTrain.shape)

In [ ]:
print(yTest)

In [ ]:
print(xTest[1])

#### STRUCTURE

In [ ]:
Model_L = Sequential()

Model_L.add(Conv2D(32,(3,3),activation="relu",input_shape=L_INPUT_SHAPE))
Model_L.add(BatchNormalization())
Model_L.add(MaxPooling2D((2,2)))

Model_L.add(Conv2D(64,(3,3),activation="relu"))
Model_L.add(Dropout(0.3))
Model_L.add(MaxPooling2D((2,2)))
Model_L.add(BatchNormalization())

Model_L.add(Conv2D(128,(3,3),activation="relu"))
Model_L.add(Dropout(0.3))
Model_L.add(MaxPooling2D((2,2)))
Model_L.add(BatchNormalization())


Model_L.add(Flatten())
Model_L.add(Dense(128,activation="relu"))
Model_L.add(BatchNormalization())
Model_L.add(Dropout(0.5))
Model_L.add(Dense(64,activation="relu"))
Model_L.add(BatchNormalization())
Model_L.add(Dropout(0.5))
Model_L.add(Dense(L_OUTPUT_CLASS,activation="softmax"))

In [ ]:
Model_L.compile(optimizer=L_COMPILE_OPTIMIZER,loss=L_COMPILE_LOSS,metrics=L_COMPILE_METRICS)

In [ ]:
L_CNN_Model = Model_L.fit(xTrain,yTrain,
                      validation_data=(xTest,yTest),
                      callbacks=Early_Stopper,
                      batch_size=12,
                      epochs=50)

In [ ]:
Model_Results = Model_L.evaluate(xTest,yTest)
print("LOSS:  " + "%.4f" % Model_Results[0])
print("ACCURACY:  " + "%.2f" % Model_Results[1])

In [ ]:
Grap_Data = pd.DataFrame(L_CNN_Model.history)
Grap_Data.plot()

#### PREDICTION

In [ ]:
Prediction_Results = Model_L.predict(xTest)

In [ ]:
Model_Test_Prediction = Prediction_Results.argmax(axis=-1)

In [ ]:
Model_True_Values = yTest.argmax(axis=-1)

In [ ]:
print(confusion_matrix(Model_True_Values,Model_Test_Prediction))

In [ ]:
print(classification_report(Model_True_Values,Model_Test_Prediction))

In [ ]:
print(Model_Test_Prediction)

In [ ]:
print(yTest[1].argmax(axis=-1))

In [ ]:
fig, axes = plt.subplots(nrows=5,
                         ncols=5,
                         figsize=(20, 20),
                        subplot_kw={'xticks': [], 'yticks': []})

for i, ax in enumerate(axes.flat):
    ax.imshow(xTest[i])
    ax.set_title(f"PREDICTION:{Model_Test_Prediction[i]}")
    ax.set_xlabel(yTest[i].argmax(axis=-1))
plt.tight_layout()
plt.show()

#### END OF THE PROJECT! THANK YOU FOR YOUR CHECKING! KEEP IT UP!